# Using tool to get information about a country, from wikipedia




## Import libaries and prepare the API credential

In [2]:
from autogen import ConversableAgent, register_function, GroupChatManager, GroupChat
from autogen import config_list_from_json

with open('../openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

config_list = config_list_from_json(env_or_file="../CONFIG_LIST", filter_dict={"model": MODEL})
config_list[0]["api_key"] = key

## A Python function to get citations. 
The input is a DOI of the source paper, and the output is a list of DOIs of the papers that the source paper has cited.


## A python function to get abstract from a DOI.
Input: DOI in string. Output: Text of the title and abstract

In [ ]:
from tools.citation_tools import *

## Agent 1: tool executor (just to wrap up the function)

In [6]:

lib_tool_executor = ConversableAgent(
    "lib_tool_executor",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    is_termination_msg=lambda msg: (msg["content"]) and ("terminate" in msg["content"].lower())
)


### Agent 2: Tool driver (caller of the function)

In [7]:
lib_tool_driver = ConversableAgent(
    "lib_tool_driver",
    system_message = "You are a helpful assistant to help users understand scientific publication. You may decide to use some external tools to search papers, with their citations, titles, abstracts, if necessary."
    "Once you received the result you needed, or you believe the result from the tools satisfy the task, say 'TERMINATE' - no need to summarize or analyze the results",
    llm_config = {"config_list": config_list},
    code_execution_config=False
)

### Register the tool to both caller and executor

*Question:* Why we need tool agents to use a tool?

In [8]:
register_function(
    get_title_abstract,
    caller = lib_tool_driver,
    executor = lib_tool_executor,
    description = "Get the title and abstract of a paper. The input is the PMID (pubmed id) of this paper"
)

register_function(
    get_pmid_from_doi,
    caller = lib_tool_driver,
    executor = lib_tool_executor,
    description = "Get the PMID (pubmed id) of a paper from its DOI as input"
)

register_function(
    get_doi_from_pmid,
    caller = lib_tool_driver,
    executor = lib_tool_executor,
    description = "Get the DOI of a paper from its PMID (pubmed id) as input"
)

register_function(
    get_pmid_from_title,
    caller = lib_tool_driver,
    executor = lib_tool_executor,
    description = "Search a paper from its title. The input is the title, and the output is the PMID of this paper"
)

register_function(
    get_citations,
    caller = lib_tool_driver,
    executor = lib_tool_executor,
    description = "Get the citations of a paper. The input is the DOI of the source paper, and the output is a list of DOIs of the papers that the source paper has cited"
)

## Make the nested agent for tools

In [9]:
from tools.tools import second_last_msg

nested_chats = [
    {
        "recipient": lib_tool_driver,
        "max_turns": 4,
        "summary_method": second_last_msg
    }
]

lib_tool_executor.register_nested_chats(
    nested_chats, 
    trigger = lambda sender: sender not in [lib_tool_driver]
)

## Agent to the read the abstract and check if the paper is about PSC or not.

In [10]:
chatbot_system_message = "You are a helpful assistant. You will receive the abstract of a scientific paper, and you judge if this paper is about a research for the PSC (Primary Sclerosing Cholangitis) disease. Please answer with simply a 'YES' or 'No' as the first line. If it is, please describe briefly what the research is about (in on sentence), If not, please give a short explanation why you don't think it is"

psc_checker = ConversableAgent(
    "psc_checker",
    system_message = chatbot_system_message,
    llm_config = {"config_list": config_list},
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [11]:
architect = ConversableAgent(
    "architect",
    system_message = "You are an AI-based architect that make plans for the whole group."
    "When you get a task, break down it into sub tasks, and each can be performed by one of your 'partner agents'."
    "You will get an introduction about what each of your partner agents can do.",
    llm_config = {"config_list": config_list},
    code_execution_config=False,  # Turn off code execution for this agent.
    human_input_mode = "ALWAYS"
)


## Now we make the group

In [12]:
lib_tool_executor.description = "I have a couple of tools to fetch paper informaiton from PubMed. I can search the DOI or PubMed Id from title, get the abstract of a paper, and find the citations of the paper"
psc_checker.description = "I can read a paragraph of English text (such as the abstract of a paper), and try to tell if the paper is about the research on the PSC disease"
architect.description = "I am the architect, and I break down a task into subtasks. I should always be the one to speak first, to make plan for sub-tasks"


group_chat = GroupChat(
    agents = [lib_tool_executor, psc_checker, architect],
    messages = [],
    max_round = 20,
    send_introductions = True
)

group_chat_manager = GroupChatManager(
    groupchat = group_chat,
    llm_config = {"config_list": config_list}
)



In [ ]:
chat_result = lib_tool_executor.initiate_chat(
    group_chat_manager,
    max_round = 6,
    message = "I have paper titled 'A biliary immune landscape map of primary sclerosing cholangitis reveals a dominant network of neutrophils and tissue-resident T cells'. Can you get all its citations, as a list DOIs. This ends the first sub-task, and then replan for the next sub-task: take the first citation (DOI), get its abstract and check if it is about PSC. Repeat this for the second and third citation."
)